### Load Rents and space from immowelt.de and House prices to get cap rates for real estate offers 

In [71]:
# Import requests and BeautifulSoup libraries
import requests
from bs4 import BeautifulSoup

# Define the URL to scrape
url = "https://www.immowelt.de/suche/jena/wohnungen/kaufen?ama=150&ami=100&d=true&sd=DESC&sf=RELEVANCE&sp=1"

# Send a GET request and get the HTML content
response = requests.get(url)
html = response.text

In [74]:
from selenium import webdriver

# Create a webdriver instance
driver = webdriver.Chrome()

# Load the website
url = "https://www.immowelt.de/suche/jena/wohnungen/kaufen?ama=150&ami=100&d=true&sd=DESC&sf=RELEVANCE&sp=1z"
driver.get(url)

# Wait for the website to load
driver.implicitly_wait(10)

# Get the HTML content of the website
html = driver.page_source

# Close the webdriver
driver.close()

In [56]:
#print(html)

In [16]:
# Find all <div> elements that have a "data-test" attribute
#listings = soup.find_all("div", attrs={"data-test": True})


In [75]:
# Import BeautifulSoup and re
from bs4 import BeautifulSoup
import re

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Find all <div> elements that have a class name that starts with "KeyFacts"
listings = soup.find_all("div", class_=re.compile("^KeyFacts"))

# Create an empty list to store the lists of information
info_list = []

# Loop through each listing
for listing in listings:
    # Find all <div> elements inside the listing that have a "data-test" attribute
    divs = listing.find_all("div", attrs={"data-test": True})
    # Create an empty list to store the information for each listing
    info = []
    # Loop through each <div> element
    for div in divs:
        # Get the text content of the <div> element
        div_text = div.text.strip()
        # Append the text to the info list
        info.append(div_text)
    # Append the info list to the info_list
    info_list.append(info)

# Print the list of lists
print(info_list)


[['325.000 €', '113 m²', '4 Zi.'], ['691.000 €', '122.8 m²', '4 Zi.'], ['656.000 €', '113.6 m²', '4 Zi.'], ['624.000 €', '113.6 m²', '4 Zi.'], ['617.000 €', '135 m²', '4.5 Zi.'], ['829.375 €', '132.7 m²', '3 Zi.'], ['440.000 €', '144 m²', '5 Zi.'], ['360.000 €', '119 m²', '4 Zi.'], ['595.000 €', '141 m²', '4 Zi.'], ['531.000 €', '113 m²', '5 Zi.'], ['365.000 €', '112 m²', '5 Zi.']]


In [77]:
# Import numpy
import numpy as np

# Create a list of lists that contains the rent and living space for each apartment listing
#rent_space_list = [['791-1.493 €', '67-139 m²'], ['1.580 €', '125.81 m²'], ['1.815 €', '134 m²'], ['1.014 €', '92.37 m²'], ['1.383 €', '117.06 m²']]
rent_space_list = info_list.copy()
# Create an empty list to store the €/m² prices
avg_list = []
rent_list = []
space_list = []

# Loop through each list in the rent_space_list
for rent_space in rent_space_list:
    # Get the rent and living space as strings
    rent_str = rent_space[0].replace(".", "")
    space_str = rent_space[1]
    print(rent_str)
    print(space_str)
    # Check if the rent or living space has a range
    if "-" in rent_str or "-" in space_str:
        # Split the strings by "-"
        rent_split = rent_str.split("-")
        #space_split = space_str.split("-")
        # Get the lower and upper bounds of the rent and living space as floats
        rent_low = float(rent_split[0])
        rent_high = float(rent_split[1][:-2]) # Remove the " €" at the end
        space_low = float(space_split[0])
        space_high = float(space_split[1][:-3]) # Remove the " m²" at the end
        # Calculate the lower and upper bounds of the €/m² price as floats
        price_low = rent_low / space_high
        price_high = rent_high / space_low
        # Calculate the average of the lower and upper bounds of the €/m² price as a float
        price_avg = (price_low + price_high) / 2
    else:
        # Get the rent and living space as floats
        rent = float(rent_str[:-2]) # Remove the " €" at the end
        space = float(space_str[:-3]) # Remove the " m²" at the end
        # Calculate the €/m² price as a float
        price_avg = rent / space
        rent_list.append(rent)
        space_list.append(space)
    avg_list.append(price_avg)

325000 €
113 m²
691000 €
122.8 m²
656000 €
113.6 m²
624000 €
113.6 m²
617000 €
135 m²
829375 €
132.7 m²
440000 €
144 m²
360000 €
119 m²
595000 €
141 m²
531000 €
113 m²
365000 €
112 m²


In [83]:
print(avg_list)
print(rent_list)
print(space_list)

[2876.1061946902655, 5627.035830618893, 5774.647887323944, 5492.957746478874, 4570.37037037037, 6250.000000000001, 3055.5555555555557, 3025.2100840336134, 4219.858156028369, 4699.115044247787, 3258.9285714285716]
[325000.0, 691000.0, 656000.0, 624000.0, 617000.0, 829375.0, 440000.0, 360000.0, 595000.0, 531000.0, 365000.0]
[113.0, 122.8, 113.6, 113.6, 135.0, 132.7, 144.0, 119.0, 141.0, 113.0, 112.0]


In [84]:
# Calculate the mean of the price_list as a float
price_mean = np.mean(avg_list)

# Print the mean of the price_list rounded to two decimals
print(f"Price per m*2 mean: {round(price_mean, 2)}")
print(f"Space mean: {round(np.mean(space_list), 2)}")
print(f"Rent mean: {round(np.mean(rent_list), 2)}")

print(f"Price per m*2 median: {round(np.median(avg_list), 2)}")
print(f"Space median: {round(np.median(space_list), 2)}")
print(f"Rent median: {round(np.median(rent_list), 2)}")

Price per m*2 mean: 4440.89
Space mean: 123.61
Rent mean: 548488.64
Price per m*2 median: 4570.37
Space median: 119.0
Rent median: 595000.0


In [57]:
# Import requests and BeautifulSoup libraries
import requests
from bs4 import BeautifulSoup

# Define the URL to scrape
url = "https://www.immowelt.de/suche/jena/haeuser/kaufen?ama=180&ami=100&d=true&sd=DESC&sf=RELEVANCE&sp=1"

# Send a GET request and get the HTML content
response = requests.get(url)
html = response.text

In [58]:
import re

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Find all <div> elements that have a class name that starts with "KeyFacts"
listings = soup.find_all("div", class_=re.compile("^KeyFacts"))

# Create an empty list to store the lists of information
info_list = []

# Loop through each listing
for listing in listings:
    # Find all <div> elements inside the listing that have a "data-test" attribute
    divs = listing.find_all("div", attrs={"data-test": True})
    # Create an empty list to store the information for each listing
    info = []
    # Loop through each <div> element
    for div in divs:
        # Get the text content of the <div> element
        div_text = div.text.strip()
        # Append the text to the info list
        info.append(div_text)
    # Append the info list to the info_list
    info_list.append(info)

# Print the list of lists
print(info_list)

[['459.000 â\x82¬', '110 mÂ²', '5 Zi.'], ['519.000 â\x82¬', '128 mÂ²', '8 Zi.'], ['440.000 â\x82¬', '160 mÂ²', '7.5 Zi.'], ['650.000 â\x82¬', '146.6 mÂ²', '5 Zi.'], ['225.000 â\x82¬', '130 mÂ²', '5 Zi.'], ['149.000 â\x82¬', '143.5 mÂ²', '4 Zi.'], ['640.000 â\x82¬', '167.48 mÂ²', '4 Zi.'], ['759.000 â\x82¬', '160 mÂ²', '6 Zi.'], ['198.000 â\x82¬', '159 mÂ²', ''], ['417.600 â\x82¬', '142.84 mÂ²', '6 Zi.'], ['849.500 â\x82¬', '126.2 mÂ²', '5 Zi.'], ['695.000 â\x82¬', '150 mÂ²', '5 Zi.'], ['459.000 â\x82¬', '130 mÂ²', '5 Zi.'], ['585.000 â\x82¬', '125 mÂ²', '5 Zi.'], ['725.000 â\x82¬', '162.12 mÂ²', '4 Zi.'], ['749.000 â\x82¬', '179 mÂ²', '7 Zi.'], ['560.000 â\x82¬', '116 mÂ²', '3 Zi.'], ['730.000 â\x82¬', '150 mÂ²', '5 Zi.'], ['599.000 â\x82¬', '155 mÂ²', '7 Zi.'], ['306.349 â\x82¬', '172.25 mÂ²', '5 Zi.']]


In [69]:
# Import numpy
import numpy as np

# Create a list of lists that contains the rent and living space for each apartment listing
#rent_space_list = [['791-1.493 €', '67-139 m²'], ['1.580 €', '125.81 m²'], ['1.815 €', '134 m²'], ['1.014 €', '92.37 m²'], ['1.383 €', '117.06 m²']]
rent_space_list = info_list.copy()
# Create an empty list to store the €/m² prices
avg_list = []
rent_list = []
space_list = []

# Loop through each list in the rent_space_list
for rent_space in rent_space_list:
    # Get the rent and living space as strings
    rent_str = rent_space[0].replace(".", "").split(" ")[0]
    space_str = rent_space[1].split(" ")[0]
    print(rent_str)
    print(space_str)
    # Check if the rent or living space has a range
    if "-" in rent_str or "-" in space_str:
        # Split the strings by "-"
        rent_split = rent_str.split("-")
        #space_split = space_str.split("-")
        # Get the lower and upper bounds of the rent and living space as floats
        rent_low = float(rent_split[0])
        rent_high = float(rent_split[1]) 
        space_low = float(space_split[0])
        space_high = float(space_split[1])
        # Calculate the lower and upper bounds of the €/m² price as floats
        price_low = rent_low / space_high
        price_high = rent_high / space_low
        # Calculate the average of the lower and upper bounds of the €/m² price as a float
        price_avg = (price_low + price_high) / 2
    else:
        # Get the rent and living space as floats
        rent = float(rent_str[:]) 
        space = float(space_str[:])
        # Calculate the €/m² price as a float
        price_avg = rent / space
        rent_list.append(rent)
        space_list.append(space)
    # Append the €/m² price to the price_list
    avg_list.append(price_avg)

459000
110
519000
128
440000
160
650000
146.6
225000
130
149000
143.5
640000
167.48
759000
160
198000
159
417600
142.84
849500
126.2
695000
150
459000
130
585000
125
725000
162.12
749000
179
560000
116
730000
150
599000
155
306349
172.25


In [70]:
# Print the mean of the price_list rounded to two decimals
print(f"Price per m*2 mean: {round(np.mean(avg_list), 2)}")
print(f"Space mean: {round(np.mean(space_list), 2)}")
print(f"Rent mean: {round(np.mean(rent_list), 2)}")

print(f"Price per m*2 median: {round(np.median(avg_list), 2)}")
print(f"Space median: {round(np.median(space_list), 2)}")
print(f"Rent median: {round(np.median(rent_list), 2)}")

Price per m*2 mean: 3724.17
Space mean: 145.65
Rent mean: 535722.45
Price per m*2 median: 4113.71
Space median: 148.3
Rent median: 572500.0
